# Notebook 1: Portfolio Optimization and Efficient Frontier

## Introduction

In this notebook, we will explore **Modern Portfolio Theory (MPT)** and construct the **Efficient Frontier**. The Efficient Frontier represents the set of optimal portfolios that offer the highest expected return for a given level of risk. By the end of this notebook, you will:

1. Load historical returns for multiple assets.
2. Compute the **covariance matrix** and **annualized returns**.
3. Construct the **Efficient Frontier**.
4. Identify the **Minimum Variance Portfolio (MVP)** and the **Maximum Sharpe Ratio Portfolio (MSR)**.
5. Compare the performance of different portfolios.

This notebook uses Python libraries such as `pandas`, `numpy`, and `matplotlib` for data analysis and visualization.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from edhec_risk_kit import *

# Load data
ffme_rets = get_ffme_returns()  # French-Fama dataset
hfi_rets = get_hfi_returns()    # EDHEC Hedge Fund Index

# Combine the datasets
rets = pd.concat([ffme_rets, hfi_rets], axis=1)
print("First 5 rows of the combined dataset:")
print(rets.head())

ModuleNotFoundError: No module named 'statsmodels'

### Explanation of the Data

- The **French-Fama dataset** contains returns for the **bottom decile (Lo 10)** and **top decile (Hi 10)** of US stocks based on market capitalization.
- The **EDHEC Hedge Fund Index** contains returns for various hedge fund strategies.
- By combining these datasets, we can create a diversified portfolio with both equity and hedge fund exposure.

---

## Step 2: Compute Annualized Returns and Covariance Matrix

To construct the Efficient Frontier, we need to compute two key inputs:

1. **Annualized Returns**: The average return of each asset, scaled to an annual basis.
2. **Covariance Matrix**: A matrix that captures the volatility of each asset and their pairwise correlations.

The **annualized return** for an asset is calculated as:

\[
\text{Annualized Return} = \left( \prod_{t=1}^{T} (1 + R_t) \right)^{\frac{n}{T}} - 1
\]

where:
- \( R_t \) is the return at time \( t \),
- \( T \) is the total number of periods,
- \( n \) is the number of periods in a year (e.g., 12 for monthly data).

The **covariance matrix** \( \Sigma \) is computed as:

$$\[
\Sigma_{ij} = \text{Cov}(R_i, R_j)
\]$$

where \( R_i \) and \( R_j \) are the returns of assets \( i \) and \( j \), respectively.

In [ ]:
# Compute annualized returns
annual_rets = annualize_rets(rets, periods_per_year=12)

# Compute covariance matrix
cov_matrix = rets.cov()

print("Annualized Returns:")
print(annual_rets)

print("\nCovariance Matrix:")
print(cov_matrix)

# Explanation of the Output

## Annualized Returns
The **annualized returns** represent the expected yearly return for each asset. This metric helps investors understand the potential growth of their investments over a one-year period.

## Covariance Matrix
The **covariance matrix** illustrates how the returns of different assets move in relation to each other. 

- **Diagonal Elements**: These represent the **variance** of each asset, which measures the asset's individual volatility.
- **Off-Diagonal Elements**: These represent the **covariance** between pairs of assets, indicating how the returns of two assets move together. A positive covariance means the assets tend to move in the same direction, while a negative covariance means they move in opposite directions.

---

# Step 3: Construct the Efficient Frontier

The **Efficient Frontier** is a set of optimal portfolios that offer the highest expected return for a given level of risk (measured by standard deviation). To construct it, we follow these steps:

1. **Generate a Range of Portfolio Weights**: We create various combinations of asset weights to form different portfolios.
2. **Compute Expected Return and Volatility**: For each portfolio, we calculate:
   - **Portfolio Return (Rp)**: The weighted average of the returns of the individual assets.
   - **Portfolio Volatility (σp)**: The standard deviation of the portfolio's returns, which measures risk.
3. **Plot the Results**: The Efficient Frontier is plotted by graphing the expected return against the portfolio volatility.

---

## Formulas

### Portfolio Return (Rp)
The portfolio return is calculated as:
$$
R_p = \sum_{i=1}^{N} w_i \cdot R_i
$$
Where:
- \( w_i \) = weight of asset \( i \)
- \( R_i \) = return of asset \( i \)
- \( N \) = total number of assets

### Portfolio Volatility (σp)
The portfolio volatility is calculated as:
$$
\sigma_p = \sqrt{w^T \Sigma w}
$$
Where:
- \( w \) = vector of asset weights
- \( \Sigma \) = covariance matrix of asset returns
- \( w^T \) = transpose of the weight vector

---

This process allows investors to identify the optimal mix of assets that maximizes returns for a given level of risk, forming the basis of modern portfolio theory.

In [ ]:
# Number of portfolios to simulate
n_portfolios = 50

# Generate the Efficient Frontier
df, ax = efficient_frontier(
    n_portfolios, rets, cov_matrix, periods_per_year=12, 
    risk_free_rate=0.03, iplot=True, hsr=True, mvp=True, ewp=True
)

# Highlight the Minimum Variance Portfolio (MVP)
mvp = df.iloc[df['volatility'].idxmin()]
print("\nMinimum Variance Portfolio (MVP):")
print(mvp)

# Highlight the Maximum Sharpe Ratio Portfolio (MSR)
msr = df.iloc[df['sharpe ratio'].idxmax()]
print("\nMaximum Sharpe Ratio Portfolio (MSR):")
print(msr)

# Plot the Efficient Frontier
plt.title("Efficient Frontier")
plt.xlabel("Volatility (Annualized)")
plt.ylabel("Return (Annualized)")
plt.show()

In [ ]:
# Compute terminal wealth for each portfolio
mvp_wealth = terminal_wealth(rets @ mvp[3:])  # MVP weights start from index 3
msr_wealth = terminal_wealth(rets @ msr[3:])  # MSR weights start from index 3
ewp_weights = np.repeat(1/len(rets.columns), len(rets.columns))  # Equally weighted
ewp_wealth = terminal_wealth(rets @ ewp_weights)

print("Terminal Wealth:")
print(f"MVP: {mvp_wealth:.2f}")
print(f"MSR: {msr_wealth:.2f}")
print(f"EWP: {ewp_wealth:.2f}")